---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [12]:
def degreeDist(g):
    degree = g.degree()
    degreeValues = sorted(set(degree.values()))
    return [list(degree.values()).count(i)/float(nx.number_of_nodes(g)) for i in degreeValues]

def graph_identification():
    result = []
    for g in P1_Graphs:
        averageClustering = nx.average_clustering(g)
        shortestPath = nx.average_shortest_path_length(g)
        degreeDistribution = degreeDist(g)
        if len(degreeDistribution) > 10:
            result.append('PA')
        elif averageClustering < 0.1:
            result.append('SW_H')
        else:
            result.append('SW_L')
    return result
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')
print(nx.info(G))
G.nodes(data=True)[:5]

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


[(0, {'Department': 1, 'ManagementSalary': 0.0}),
 (1, {'Department': 1, 'ManagementSalary': nan}),
 (2, {'Department': 21, 'ManagementSalary': nan}),
 (3, {'Department': 21, 'ManagementSalary': 1.0}),
 (4, {'Department': 21, 'ManagementSalary': 1.0})]

### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [5]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC

def salary_predictions():
    def check(node):
        salary = node[1]['ManagementSalary']
        if salary == 0:
            return 0
        elif salary == 1:
            return 1
        else:
            return None

    df = pd.DataFrame(index=G.nodes())
    df['cluster'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    df['closenessCentrality'] = pd.Series(nx.closeness_centrality(G, normalized=True))
    df['betweenessCentrality'] = pd.Series(nx.betweenness_centrality(G, normalized=True))
    df['degreeCentrality'] = pd.Series(nx.degree_centrality(G))
    df['pageRank'] = pd.Series(nx.pagerank(G))
    df['check'] = pd.Series([check(item) for item in G.nodes(data=True)])
    
    dfTrain = df[~pd.isnull(df['check'])]
    dfTest = df[pd.isnull(df['check'])]
    operations = ['cluster', 'degree', 'closenessCentrality', 'betweenessCentrality', 'degreeCentrality', 'pageRank']
    
    X_train = dfTrain[operations]
    Y_train = dfTrain['check']
    X_test = dfTest[operations]
    
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    mlpc = MLPClassifier(hidden_layer_sizes = [10, 5], alpha = 5, random_state = 0, solver='lbfgs')
    mlpc.fit(X_train_scaled, Y_train)
    prediction = mlpc.predict_proba(X_test_scaled)[:, 1]
    return pd.Series(prediction,X_test.index)

salary_predictions()

1       0.128470
2       0.586087
5       0.979631
8       0.133719
14      0.304638
18      0.208705
27      0.269559
30      0.336395
31      0.169328
34      0.143550
37      0.096233
40      0.195147
45      0.162242
54      0.250134
55      0.252809
60      0.177006
62      0.993631
65      0.550121
77      0.078464
79      0.132164
97      0.099783
101     0.057493
103     0.214433
108     0.122917
113     0.454702
122     0.044327
141     0.396130
142     0.863942
144     0.072022
145     0.420065
          ...   
913     0.045689
914     0.078507
915     0.002083
918     0.144313
923     0.035128
926     0.090037
931     0.055935
934     0.013913
939     0.011129
944     0.002347
945     0.039563
947     0.162650
950     0.098187
951     0.038277
953     0.027317
959     0.002916
962     0.002434
963     0.139918
968     0.110188
969     0.104585
974     0.094999
984     0.012649
987     0.110327
989     0.113273
991     0.116860
992     0.003948
994     0.003071
996     0.0024

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [6]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler

def new_connections_predictions():
    for node in G.nodes():
        G.node[node]['community'] = G.node[node]['Department']
        
    pa = list(nx.preferential_attachment(G))
    df = pd.DataFrame(index=[(item[0], item[1]) for item in pa])
    df['pa'] = [item[2] for item in pa]
    
    cnSoundHopcroft = list(nx.cn_soundarajan_hopcroft(G))
    dfCnSoundHopcroft = pd.DataFrame(index=[(item[0], item[1]) for item in cnSoundHopcroft])
    dfCnSoundHopcroft['cnSoundHopcroft'] = [item[2] for item in cnSoundHopcroft]
    df = df.join(dfCnSoundHopcroft, how='outer')
    df['cnSoundHopcroft'] = df['cnSoundHopcroft'].fillna(value=0)
    
    df['rai'] = [item[2] for item in list(nx.resource_allocation_index(G))]
    df['jaccardCoe'] = [item[2] for item in list(nx.jaccard_coefficient(G))]
    
    df = future_connections.join(df, how='outer')
    dfTrain = df[~pd.isnull(df['Future Connection'])]
    dfTest = df[pd.isnull(df['Future Connection'])]
    features = ['cnSoundHopcroft', 'pa', 'rai', 'jaccardCoe']
    
    X_train = dfTrain[features]
    Y_train = dfTrain['Future Connection']
    X_test = dfTest[features]
    
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    mlcp = MLPClassifier(hidden_layer_sizes = [10, 5], alpha = 5, random_state = 0, solver='lbfgs')
    mlcp.fit(X_train_scaled, Y_train)
    
    test = mlcp.predict_proba(X_test_scaled)[:, 1]
    prediction = pd.Series(test,X_test.index)
    result = future_connections[pd.isnull(future_connections['Future Connection'])]
    result['predict'] = [prediction[item] for item in result.index]
    return result['predict']

new_connections_predictions()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(107, 348)    0.029287
(542, 751)    0.011763
(20, 426)     0.586093
(50, 989)     0.011841
(942, 986)    0.011925
(324, 857)    0.011831
(13, 710)     0.146574
(19, 271)     0.128051
(319, 878)    0.011868
(659, 707)    0.011771
(49, 843)     0.011893
(208, 893)    0.011790
(377, 469)    0.008825
(405, 999)    0.019559
(129, 740)    0.018294
(292, 618)    0.019216
(239, 689)    0.011874
(359, 373)    0.009982
(53, 523)     0.079868
(276, 984)    0.011889
(202, 997)    0.011905
(604, 619)    0.055465
(270, 911)    0.011873
(261, 481)    0.068997
(200, 450)    0.998463
(213, 634)    0.011739
(644, 735)    0.039804
(346, 553)    0.011545
(521, 738)    0.011174
(422, 953)    0.018574
                ...   
(672, 848)    0.011873
(28, 127)     0.978079
(202, 661)    0.011512
(54, 195)     0.999381
(295, 864)    0.011818
(814, 936)    0.011751
(839, 874)    0.011925
(139, 843)    0.011825
(461, 544)    0.010917
(68, 487)     0.010872
(622, 932)    0.011800
(504, 936)    0.017512
(479, 528) 